In [78]:
import torch
import pandas as pd
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, Subset
from torchvision import transforms, models
import os
from PIL import Image
import numpy as np

# Load the model with map_location set to CPU
model_path = 'resnet50_0.497.pkl'

from torchvision.models import resnet50

# Initialize a ResNet50 model
model_resnet50 = resnet50()

# Modify the fully connected layer to match the custom model's output size
model_resnet50.fc = torch.nn.Linear(in_features=2048, out_features=102)

# Load the model parameters
model_resnet50.load_state_dict(torch.load(model_path, map_location='cpu'))

# Set the model to evaluation mode
model_resnet50.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [79]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torch.utils.tensorboard import SummaryWriter
from torchvision import transforms, models
import os
from PIL import Image
import numpy as np
import os

In [80]:
torch.autograd.set_detect_anomaly(True)

In [81]:
class CustomDataset(Dataset):
    def __init__(self, txt_file, img_dir, transform=None):
        self.img_labels = np.loadtxt(txt_file, dtype=str)
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_name = os.path.join(self.img_dir, self.img_labels[idx, 0])
        image = Image.open(img_name).convert('RGB')
        label = int(self.img_labels[idx, 1])
        if self.transform:
            image = self.transform(image)
        return image, label

In [82]:

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
train_dataset = CustomDataset(txt_file='train.txt', img_dir='images', transform=transform)
val_dataset = CustomDataset(txt_file='val.txt', img_dir='images', transform=transform)
test_dataset = CustomDataset(txt_file='test.txt', img_dir='images', transform=transform)

# create 60% percentage of subset dataset
# train_subset_indices = list(range(len(train_dataset_full)))
# test_subset_indices = list(range(len(test_dataset_full)))
# np.random.shuffle(train_subset_indices)
# np.random.shuffle(test_subset_indices)
# train_subset_indices = train_subset_indices[:int(0.6* len(train_subset_indices))]
# test_subset_indices = test_subset_indices[:int(0.6* len(test_subset_indices))]

# train_dataset = Subset(train_dataset_full, train_subset_indices)
# test_dataset = Subset(test_dataset_full, test_subset_indices)

train_loader = DataLoader(train_dataset, batch_size=64, num_workers=4, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, num_workers=4, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=64, num_workers=4, shuffle=False)

In [83]:
# Load the model architecture
model = models.resnet50(pretrained=False)

writer = SummaryWriter()

for param in model.parameters():
    param.requires_grad = False
# Adjust the final fully connected layer to match the number of classes
num_classes = 102
model.fc = nn.Linear(model.fc.in_features, num_classes)

# Load the pre-trained weights while ignoring the final layer
pretrained_dict = torch.load('resnet50_0.497.pkl', map_location=torch.device('cpu'))
# pretrained_dict = torch.load('resnet50_0.497.pkl')

model_dict = model.state_dict()

# Filter out unnecessary keys
pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in model_dict and v.size() == model_dict[k].size()}
model_dict.update(pretrained_dict)

# Load the new state dict
model.load_state_dict(model_dict)

# Move the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

criterion = nn.CrossEntropyLoss()
# optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
# optimizer = torch.optim.AdamW(model.classifier.parameters(), lr=0.0001)  


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [84]:
# Define sparse regularization loss
# def sparse_loss(model, lambda_s=1e-5):
#     loss = 0.0
#     for param in model.parameters():
#         loss += lambda_s * torch.sum(torch.abs(param))
#     return loss
def sparse_loss(model, images):
    loss = 0
    values = images
    for module in model.children():
        if isinstance(module, nn.Linear):
            values = torch.flatten(values, start_dim=1)
        values = module(values)
        # 确保激活函数不是就地操作
        if hasattr(module, 'activation'):
            values = module.activation(values)
        loss += torch.mean(torch.abs(values))
    return loss
# Define cutmix function
def cutmix(data, targets, alpha=1.0):
    indices = torch.randperm(data.size(0))
    shuffled_data = data[indices]
    shuffled_targets = targets[indices]
    lam = np.clip(np.random.beta(alpha, alpha), 0.3, 0.4)
    bbx1, bby1, bbx2, bby2 = rand_bbox(data.size(), lam)
    data[:, :, bbx1:bbx2, bby1:bby2] = shuffled_data[:, :, bbx1:bbx2, bby1:bby2]
    targets_a, targets_b = targets, shuffled_targets
    return data, targets_a, targets_b, lam

def rand_bbox(size, lam):
    W = size[2]
    H = size[3]
    cut_rat = np.sqrt(1. - lam)
    cut_w = int(W * cut_rat)
    cut_h = int(H * cut_rat)

    cx = np.random.randint(W)
    cy = np.random.randint(H)

    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)

    return bbx1, bby1, bbx2, bby2

In [86]:
from torch.optim.lr_scheduler import StepLR
import time

In [87]:
import time
optimizer = torch.optim.AdamW(model.parameters(), lr=0.0001)
scheduler= StepLR(optimizer, step_size=50, gamma=0.1)  # 学习率调度器
def train(model, train_loader, criterion, optimizer, epoch, alpha=1.0): 
        start_time = time.time()
        model.train()
        total_loss = 0
        total = correct = 0
        for batch_idx, (images, labels) in enumerate(train_loader):
            images, labels = images.to(device), labels.to(device)

            if np.random.rand() < 0.5:  # 50% 概率应用 CutMix
                images, targets_a, targets_b, lam = cutmix(images, labels, alpha)
                outputs = model(images)
                loss = lam * criterion(outputs, targets_a) + (1 - lam) * criterion(outputs, targets_b)
            else:
                outputs = model(images)
                loss = criterion(outputs, labels)

            # 计算稀疏正则化损失
            # reg_loss = sparse_loss(model, images)
            # total_loss = loss + 0.01 * reg_loss  # 添加稀疏正则化损失
            total_loss = loss
            
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            optimizer.zero_grad()
            total_loss.backward()
            optimizer.step()
            accuracy = 100 * correct / total
            print(f'Epoch {epoch + 1}, Loss: {total_loss.item():.4f}, Accuracy: {accuracy:.2f}%')
            writer.add_scalar('Loss/train', total_loss, epoch + 1)
            writer.add_scalar('Accuracy/train', accuracy, epoch + 1)
            # if (batch_idx + 1) % 10 == 0:
            #     accuracy = 100 * correct / total
                # print(f'Epoch {epoch + 1}, Batch {batch_idx + 1}, Loss: {total_loss.item():.4f}, Accuracy: {accuracy:.2f}%')
                # writer.add_scalar('training loss', total_loss.item(), epoch * len(train_loader) + batch_idx)

        elapsed_time = time.time() - start_time
        print(f'Epoch {epoch + 1} completed in {elapsed_time:.2f} seconds')

In [90]:

# Evaluation 
def evaluate(model, loader, criterion, mode):
    model.eval()
    correct = 0
    total = 0
    running_loss = 0.0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    avg_loss = running_loss / len(loader)
    writer.add_scalar(f'{mode} Accuracy', accuracy, epoch)
    writer.add_scalar(f'{mode} Loss', avg_loss, epoch)
    return accuracy, avg_loss    

num_epochs = 10
for epoch in range(num_epochs):
    train_loss = train(model, train_loader, criterion, optimizer, epoch)
    scheduler.step()  # Update learning rate  
    val_accuracy, val_loss = evaluate(model, val_loader, criterion, mode='Validation')
    print(f'Epoch {epoch+1}/{num_epochs}, Validation Accuracy: {val_accuracy:.2f}%, Validation Loss: {val_loss:.4f}')
# Evaluate the model on the test set
test_accuracy, test_loss = evaluate(model, test_loader, criterion, mode='Test')
print(f'Test Accuracy: {test_accuracy:.2f}%, Test Loss: {test_loss:.4f}')
writer.close()

Epoch 1, Loss: 3.9479, Accuracy: 29.69%
Epoch 1, Loss: 4.9461, Accuracy: 47.66%
Epoch 1, Loss: 3.5712, Accuracy: 42.19%
Epoch 1, Loss: 4.2487, Accuracy: 42.19%
Epoch 1, Loss: 3.5743, Accuracy: 35.31%
Epoch 1, Loss: 3.6258, Accuracy: 30.73%
Epoch 1, Loss: 5.0842, Accuracy: 35.94%
Epoch 1, Loss: 3.6246, Accuracy: 32.81%
Epoch 1, Loss: 1.1500, Accuracy: 37.33%
Epoch 1, Loss: 0.8567, Accuracy: 41.56%
Epoch 1, Loss: 1.3202, Accuracy: 44.60%
Epoch 1, Loss: 5.1451, Accuracy: 45.57%
Epoch 1, Loss: 3.7410, Accuracy: 43.03%
Epoch 1, Loss: 3.9444, Accuracy: 41.52%
Epoch 1, Loss: 1.2728, Accuracy: 43.44%
Epoch 1, Loss: 4.2936, Accuracy: 43.75%
Epoch 1, Loss: 3.3810, Accuracy: 41.73%
Epoch 1, Loss: 3.9565, Accuracy: 41.84%
Epoch 1, Loss: 5.4874, Accuracy: 42.19%
Epoch 1, Loss: 4.5947, Accuracy: 42.42%
Epoch 1, Loss: 0.7195, Accuracy: 44.42%
Epoch 1, Loss: 0.8390, Accuracy: 45.88%
Epoch 1, Loss: 3.4897, Accuracy: 44.63%
Epoch 1, Loss: 0.6836, Accuracy: 46.29%
Epoch 1, Loss: 4.2763, Accuracy: 46.44%
